In [17]:
from selenium import webdriver
import bs4
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import pandas as pd
import requests
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

In [18]:
# Set the path to your Chrome WebDriver executable
chrome_driver_path = r'C:\Users\MSI\jupyter python\web_scraping/chromedriver2.exe'

In [19]:
# open Chrome WebDriver
service = Service(executable_path=chrome_driver_path)
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)

In [20]:
# open Jobthai browser by Selenium
# driver.get('https://www.jobthai.com/%E0%B8%AB%E0%B8%B2%E0%B8%87%E0%B8%B2%E0%B8%99')

# click <IT เงินเดือน 50k+>
# it_button1 = driver.find_element("xpath", "/html/body/div/section/main/div[1]/div/div/div/div[2]/div/div[1]/div/div[2]/a[2]/h2")
# it_button1.click()

In [21]:
# Create an empty DataFrame to store the data
df1 = pd.DataFrame(columns=["ID", "Location"])

In [22]:
job_titles_list = []
company_name_list = []
salary_list = []
date_list = []
job_details_list = [] 
job_properties_list = []

for page in range(1, 12):
    url = f'https://www.jobthai.com/th/jobs?jobtype=7&salarymin=50000&orderBy=SALARY_DESC&page={page}'
    driver.get(url)
    data = driver.page_source
    soup = bs4.BeautifulSoup(data)
    
    job_titles = soup.find_all('h2', {'class':'ohgq7e-0 elGPWd'})
    company_name = soup.find_all('span', {'class': 'msklqa-2 icuLsB'})
    salary = soup.find_all('span', {'id': 'salary-text', 'class': 'ohgq7e-0 msklqa-5 fzpspm'})
    date = soup.find_all('div', {'style': 'text-align:right'})
    
    for titles in job_titles:
        job_titles_list.append(titles.text)
    for company in company_name:
        company_name_list.append(company.text)
    company_name_list = list(filter(None,company_name_list))
    for salary_i in salary:
        salary_list.append(salary_i.text) 
    for date_i in date:
        date_list.append(date_i.text)
    
    # Extract job elements as before for location
    job_elements = soup.find_all('div', class_=['ant-row msklqa-8 crLDlD', 'ant-row msklqa-8 iljQQs'])

    for job_element in job_elements:
        company_id = job_element.find('span', class_='ohgq7e-0 msklqa-6 gRACqi').text
        location_elements = job_element.find_all(class_=["ohgq7e-0 sc-2q8vn1-2 gcHgCC", "ohgq7e-0 JGBCM"])

        # Check if the company has locations
        if location_elements:
            locations = [location.text for location in location_elements]

            # Append the data to the DataFrame
            for location in locations:
                df1 = df1.append({"ID": company_id, "Location": location}, ignore_index=True)
    
    for job_element in job_elements:
        # Extract the company link
        company_link = job_element.find('a', href=True)['href']
        company_url = f'https://www.jobthai.com{company_link}'

        # Visit the company's page
        driver.get(company_url)
        
        # Find job detail links on the company's page
        job_detail_links = driver.find_elements(By.CSS_SELECTOR, 'a[id^="job-detail-"]')
        
        for job_detail_link in job_detail_links:
            job_detail_url = job_detail_link.get_attribute('href')
            
            # Visit the job detail page
            driver.get(job_detail_url)
            job_data = driver.page_source
            job_soup = bs4.BeautifulSoup(job_data, 'html.parser')
         
        
            """
            job_detail = job_soup.find_all('span', {'id':'job-detail'})
            job_properties = job_soup.find_all('div', {'id':'job-properties-wrapper'})
            
            for detail in job_detail:
                job_details_list.append(detail.text)
                
            for prop in job_properties:
                job_properties_list.append(prop.text)
            """  
            # Try to find job details and properties, and handle if not found
            try:
                job_detail = job_soup.find('span', {'id': 'job-detail'}).text
            except AttributeError:
                job_detail = None

            try:
                job_properties = job_soup.find('div', {'id': 'job-properties-wrapper'}).text
            except AttributeError:
                job_properties = None

            job_details_list.append(job_detail)
            job_properties_list.append(job_properties)

In [23]:
# create a list of data
info_df = pd.DataFrame([job_details_list, job_properties_list])

In [24]:
info_df = info_df.transpose()

# set pd
pd.set_option('display.max_colwidth', 5000)
pd.set_option('max_rows', None)

# set column name
info_df.columns = ['job_details', 'job_properties']
info_df

,job_details,job_properties
0,"Primary Responsibilities\n• Using agile methodology values, principles, and practices to plan, manage, and deliver solutions.\n• Training, mentoring, and supporting scrum teams to follow agile values, principles, and practices.\n• Determining and managing tasks, issues, risks, and action items.\n• Scheduling and facilitating scrum events, meetings, and decision-making processes.\n• Monitoring progress and performance and helping teams to make improvements.\n• Planning and organizing demos and product/system testing.\n• Ensuring the proper use of collaborative processes and removing impediments for the scrum team.\n• Tracking project processes, deliverables, and financials.\n• Preparing and presenting status reports to stakeholders.\n• ""Scrum master certification is required""\n\nJob Requirements\n• Scrum Master Requirements:\n• Bachelor's degree in computer science, information technology, or a relevant field preferred.\n• Certified scrum master preferred.\n• 2+ years of experience as a scrum master or in a similar role.\n• Working knowledge of agile methodology, techniques, and frameworks, such as Scrum or Kanban, to deliver solutions.\n• Leadership and management experience.\n• Strong communication and presentation skills.\n• Strong analytical and problem-solving skills.\n• can communicate in English effectively (reports to US boss)\n• can Zoom with US in different time zone from time to time (around twice a week)\n• have worked for any startup or tech companies\n• Able to speak English","คุณสมบัติผู้สมัคร Bachelor's degree in computer science, information technology, or a relevant field preferred. 2+ years of experience as a scrum master or in a similar role. Certified scrum master preferred. Working knowledge of agile methodology, techniques, and frameworks, such as Scrum or Kanban, to deliver solutions. Leadership and management experience. Able to speak English"
1,"Job Description\n-ออกแบบและเขียน API เพื่อรองรับการเรียกใช้งานของ application และ web frontend\n-ปรับปรุงประสิทธิภาพของ code ให้ทำงานได้รวดเร็วยิ่งขึ้น\n-เขียนและแก้ไข code ตามที่ได้รับ requirement\n\nSpec ที่เพิ่มความเป็น superman\n-สามารถเขียน Nodejs และเข้าใจถึง NPM\n-รู้วิธีการใช้ฐานข้อมูลประเภท NoSQL\n-รู้จัก หรือเคยลองจีบ vuejs, angular js, react\n-รู้จัก ionic framework, capacitorjs จากพื้นฐานไปจนถึงร่วมหอกันแล้ว\n\n\nลักษณะนิสัยที่มองหา\n-เป็นคนที่กล้ามีความเห็นแย้ง และรู้วิธีสื่อสารแบบไม่กร้าวร้าว (คนตรงไม่ใช่คนกร้าวร้าว อย่าเข้าใจผิด)\n-กล้าแชร์สิ่งที่เจอใหม่ๆ กับทีมหรือหัวหน้างาน\n-อ่านเก่ง อ่านไปทุกเรื่อง รู้เยอะรู้กว้างยิ่งดี\n-ไม่มีนิสัยเดี๋ยวค่อยมาแก้ หรือเอาแบบนี้ไปก่อนกับการ coding\n-ไม่มีความคิดว่า Developer คือพระเจ้าจะใส่อารมณ์กับใครก็ได้\n\n\n---------\nสิ่งที่ CEO (K.Danny) สนใจเป็นพิเศษในตัว Candidate ทุกตำแหน่ง: \n\n- #ไม่ได้สนวุฒิการศึกษาว่าต้องสูงแค่ไหนอย่างไร เพราะการศึกษาแท้จริงอยู่ที่ปลายนิ้ว วันนี้อยากเรียน Calculus ก็เรียนได้ พรุ่งนี้จะเปลี่ยนเป็น Digital marketing ก็แค่เปลี่ยน keyword คำค้นหา \n- ไม่ยอมแพ้อะไรง่ายๆ อะไรที่คนพูดว่าไม่ได้ ยิ่งต้องทำมันให้ได้ \n- กล้าคิด กล้าทำ ทำงานแบบไม่กลัวโดนนายด่า \n- บ้านๆ ง่ายๆ ไม่คิดเยอะ ไม่หยุมหยิม คุยกันเหมือนสมัยเพื่อน ม. ปลาย ไม่ใส่หน้ากากเข้าหากัน \n- เป็นผู้ชาย เล่นเวท เพราะจะพูดภาษาเดียวกัน (K.Danny เคารพเพศหญิง เทิดทูนภรรยา และรักลูกสาวมาก ไม่มีเจตนาเหยียดเพศ) \n- มีความเข้าใจคณิตพื้นฐานเช่น พีชคณิต, ความน่าจะเป็น, สถิติ, Log/เลขยกกำลัง, Calculus ในระดับเบื้องต้น - สามารถเขียนโปรแกรมในภาษาใดก็ได้หรือเป็นผู้ใช้ Google Sheet (Excel) ระดับสูง \n- มีพลังงานบวก ไม่คิดลบ ความเป็นลูกผู้ชาย เกิน 100% #ไม่นินทาคน \n***ยอมลาออกไปตายดาบหน้า ดีกว่าอยู่ในองค์กรที่มีพลังงานเฉื่อย (เช่น นั่งนับวันลาป่วยว่า ปีนี้ป่วยครบสิทธิ์หรือยัง) ……. ปีนึงกฏหมายให้ป่วย 30วัน เจอคนเฉื่อย ปีนึงทำงาน 250วัน พี่แก ซัดไป 30/250 (12%.... โอ้ววว ...)","คุณสมบัติผู้สมัคร สามารถเขียนโปรแกรมด้วยภาษา PHP, HTML, Javascript, CSS ได้อย่างถูกต้อง เข้าใจโครงสร้างการเขียนโปรแกรมแบบ MVC และ HMVC รู้ถึงวิธีการเขียนโปรแกรมให้มีความซับซ้อนต่ำ แต่ประสิทธิภาพสูงภายใต้ performance ที่จำกัดของเครื่อง ser

In [25]:
# Clean job_details column
info_df['job_details'] = info_df['job_details'].str.strip()  # Remove leading/trailing whitespace
info_df['job_details'] = info_df['job_details'].str.replace(r'\n', ' / ', regex=True)  # Replace newline characters ("\n") with forward slash ("/")
info_df['job_details'] = info_df['job_details'].str.replace(r'•', '')  # Remove bullet points

# Clean job_properties column
info_df['job_properties'] = info_df['job_properties'].str.strip()  
info_df['job_properties'] = info_df['job_properties'].str.replace(r'คุณสมบัติผู้สมัคร', '')  # Remove the specified string
info_df['job_properties'] = info_df['job_properties'].str.replace(r'•', '')  # Remove bullet points
info_df['job_properties'] = info_df['job_properties'].str.replace(r'\n', ' / ', regex=True)  # Replace newline characters ("\n") with forward slash ("/")

# Create the new DataFrame clean_info
clean_info = info_df.copy()

In [26]:
clean_info 

,job_details,job_properties
0,"Primary Responsibilities / Using agile methodology values, principles, and practices to plan, manage, and deliver solutions. / Training, mentoring, and supporting scrum teams to follow agile values, principles, and practices. / Determining and managing tasks, issues, risks, and action items. / Scheduling and facilitating scrum events, meetings, and decision-making processes. / Monitoring progress and performance and helping teams to make improvements. / Planning and organizing demos and product/system testing. / Ensuring the proper use of collaborative processes and removing impediments for the scrum team. / Tracking project processes, deliverables, and financials. / Preparing and presenting status reports to stakeholders. / ""Scrum master certification is required"" / / Job Requirements / Scrum Master Requirements: / Bachelor's degree in computer science, information technology, or a relevant field preferred. / Certified scrum master preferred. / 2+ years of experience as a scrum master or in a similar role. / Working knowledge of agile methodology, techniques, and frameworks, such as Scrum or Kanban, to deliver solutions. / Leadership and management experience. / Strong communication and presentation skills. / Strong analytical and problem-solving skills. / can communicate in English effectively (reports to US boss) / can Zoom with US in different time zone from time to time (around twice a week) / have worked for any startup or tech companies / Able to speak English","Bachelor's degree in computer science, information technology, or a relevant field preferred. 2+ years of experience as a scrum master or in a similar role. Certified scrum master preferred. Working knowledge of agile methodology, techniques, and frameworks, such as Scrum or Kanban, to deliver solutions. Leadership and management experience. Able to speak English"
1,"Job Description / -ออกแบบและเขียน API เพื่อรองรับการเรียกใช้งานของ application และ web frontend / -ปรับปรุงประสิทธิภาพของ code ให้ทำงานได้รวดเร็วยิ่งขึ้น / -เขียนและแก้ไข code ตามที่ได้รับ requirement / / Spec ที่เพิ่มความเป็น superman / -สามารถเขียน Nodejs และเข้าใจถึง NPM / -รู้วิธีการใช้ฐานข้อมูลประเภท NoSQL / -รู้จัก หรือเคยลองจีบ vuejs, angular js, react / -รู้จัก ionic framework, capacitorjs จากพื้นฐานไปจนถึงร่วมหอกันแล้ว / / / ลักษณะนิสัยที่มองหา / -เป็นคนที่กล้ามีความเห็นแย้ง และรู้วิธีสื่อสารแบบไม่กร้าวร้าว (คนตรงไม่ใช่คนกร้าวร้าว อย่าเข้าใจผิด) / -กล้าแชร์สิ่งที่เจอใหม่ๆ กับทีมหรือหัวหน้างาน / -อ่านเก่ง อ่านไปทุกเรื่อง รู้เยอะรู้กว้างยิ่งดี / -ไม่มีนิสัยเดี๋ยวค่อยมาแก้ หรือเอาแบบนี้ไปก่อนกับการ coding / -ไม่มีความคิดว่า Developer คือพระเจ้าจะใส่อารมณ์กับใครก็ได้ / / / --------- / สิ่งที่ CEO (K.Danny) สนใจเป็นพิเศษในตัว Candidate ทุกตำแหน่ง: / / - #ไม่ได้สนวุฒิการศึกษาว่าต้องสูงแค่ไหนอย่างไร เพราะการศึกษาแท้จริงอยู่ที่ปลายนิ้ว วันนี้อยากเรียน Calculus ก็เรียนได้ พรุ่งนี้จะเปลี่ยนเป็น Digital marketing ก็แค่เปลี่ยน keyword คำค้นหา / - ไม่ยอมแพ้อะไรง่ายๆ อะไรที่คนพูดว่าไม่ได้ ยิ่งต้องทำมันให้ได้ / - กล้าคิด กล้าทำ ทำงานแบบไม่กลัวโดนนายด่า / - บ้านๆ ง่ายๆ ไม่คิดเยอะ ไม่หยุมหยิม คุยกันเหมือนสมัยเพื่อน ม. ปลาย ไม่ใส่หน้ากากเข้าหากัน / - เป็นผู้ชาย เล่นเวท เพราะจะพูดภาษาเดียวกัน (K.Danny เคารพเพศหญิง เทิดทูนภรรยา และรักลูกสาวมาก ไม่มีเจตนาเหยียดเพศ) / - มีความเข้าใจคณิตพื้นฐานเช่น พีชคณิต, ความน่าจะเป็น, สถิติ, Log/เลขยกกำลัง, Calculus ในระดับเบื้องต้น - สามารถเขียนโปรแกรมในภาษาใดก็ได้หรือเป็นผู้ใช้ Google Sheet (Excel) ระดับสูง / - มีพลังงานบวก ไม่คิดลบ ความเป็นลูกผู้ชาย เกิน 100% #ไม่นินทาคน / ***ยอมลาออกไปตายดาบหน้า ดีกว่าอยู่ในองค์กรที่มีพลังงานเฉื่อย (เช่น นั่งนับวันลาป่วยว่า ปีนี้ป่วยครบสิทธิ์หรือยัง) ……. ปีนึงกฏหมายให้ป่วย 30วัน เจอคนเฉื่อย ปีนึงทำงาน 250วัน พี่แก ซัดไป 30/250 (12%.... โอ้ววว ...)","สามารถเขียนโปรแกรมด้วยภาษา PHP, HTML, Javascript, CSS ได้อย่างถูกต้อง เข้าใจโครงสร้างการเขียนโปรแกรมแบบ MVC และ HMVC รู้ถึงวิธีการเขียนโปรแกรมให้มีความซับซ้อนต่ำ แต่ประสิทธิภาพสูงภายใต้ performance ที่จำกัดของเครื่อง server รู้จักระบบปฏิบัติการตระกูล Linux เป็น

In [27]:
# Group by 'ID' and aggregate 'Location' values into a list
result_df = df1.groupby('ID')['Location'].apply(list).reset_index()
result_df['Location'] = result_df['Location'].apply(lambda x: ', '.join(x))

# Convert the "ID" column to numeric type to ensure proper sorting
result_df['ID'] = result_df['ID'].replace('\.', '', regex=True).astype(int)

# Sort the DataFrame by the "ID" column in ascending order
result_df = result_df.sort_values(by='ID')

# Reset the index of the sorted DataFrame
loc_df = result_df.reset_index(drop=True)
loc_df.head()

,ID,Location
0,1,เขตดินแดง กรุงเทพมหานคร
1,2,เขตลาดพร้าว กรุงเทพมหานคร
2,3,นิคมอุตสาหกรรมดับบลิวเอชเอ ชลบุรี 2 จ.ชลบุรี
3,4,นิคมอุตสาหกรรมดับบลิวเอชเอ ชลบุรี 2 จ.ชลบุรี
4,5,เขตประเวศ กรุงเทพมหานคร


In [28]:
# create a list of data
df = pd.DataFrame([job_titles_list, company_name_list, salary_list, date_list])

In [29]:
df = df.transpose()
# set column name
df.columns = ['job_titles', 'company_name', 'salary_offer', 'hiring_date']

# set pd
pd.set_option('display.max_colwidth', 100)
pd.set_option('max_rows', None)

# join dataframe
job_df1 = df.join(loc_df, lsuffix='_df', rsuffix='_loc')
job_df1

,job_titles,company_name,salary_offer,hiring_date,ID,Location
0,Scrum Master (พนักงานประจำ) AAC,ASCGroup,"100,000",19 ต.ค. 66,1,เขตดินแดง กรุงเทพมหานคร
1,Full Stack Developer,"FITWHEY Co.,Ltd.","90,000 THB",17 ต.ค. 66,2,เขตลาดพร้าว กรุงเทพมหานคร
2,Digital Product Quality Analyst/ Remote/ English Speaking,บริษัท ฟินคับ จำกัด,"90,000 - 120,000 THB",18 ต.ค. 66,3,นิคมอุตสาหกรรมดับบลิวเอชเอ ชลบุรี 2 จ.ชลบุรี
3,Senior Software Engineer (Full Stack)/ Remote/ English Speaking,บริษัท ฟินคับ จำกัด,"90,000 - 120,000 THB",18 ต.ค. 66,4,นิคมอุตสาหกรรมดับบลิวเอชเอ ชลบุรี 2 จ.ชลบุรี
4,Full Stack Developer (.Net & React.js),บริษัท มอตโต้ อ๊อคชั่น (ประเทศไทย) จำกัด,"85,000",12 ต.ค. 66,5,เขตประเวศ กรุงเทพมหานคร
5,Java Developer,บริษัท ไอ แพสชั่น จำกัด,"Up to 85,000 บาท",19 ต.ค. 66,6,"MRT รัชดาภิเษก, สุทธิสาร"
6,Project Coordinator-AOS,ASCGroup,"80,000",19 ต.ค. 66,7,เขตจตุจักร กรุงเทพมหานคร
7,C#.Net Programmer (สมุทรสาคร),"Nemera Technologies Co, Ltd.","75,000",12 ต.ค. 66,8,อ.เมืองสมุทรสาคร จ.สมุทรสาคร
8,Technical Support Manager,"40 Hours recruitment (Thailand) Co., Ltd.","75,000 - 90,000",18 ต.ค. 66,9,เขตดินแดง กรุงเทพมหานคร
9,System Analyst,"THE PRODIGY (THAILAND) CO., LTD.","70,000 - 120,000 THB",12 ต.ค. 66,10,เขตคลองเตย กรุงเทพมหานคร


In [30]:
# join dataframe
job_df2 = job_df1.join(info_df, lsuffix='_df', rsuffix='_loc')
job_df2

,job_titles,company_name,salary_offer,hiring_date,ID,Location,job_details,job_properties
0,Scrum Master (พนักงานประจำ) AAC,ASCGroup,"100,000",19 ต.ค. 66,1,เขตดินแดง กรุงเทพมหานคร,"Primary Responsibilities / Using agile methodology values, principles, and practices to plan, m...","Bachelor's degree in computer science, information technology, or a relevant field preferred. ..."
1,Full Stack Developer,"FITWHEY Co.,Ltd.","90,000 THB",17 ต.ค. 66,2,เขตลาดพร้าว กรุงเทพมหานคร,Job Description / -ออกแบบและเขียน API เพื่อรองรับการเรียกใช้งานของ application และ web frontend ...,"สามารถเขียนโปรแกรมด้วยภาษา PHP, HTML, Javascript, CSS ได้อย่างถูกต้อง เข้าใจโครงสร้างการเขียน..."
2,Digital Product Quality Analyst/ Remote/ English Speaking,บริษัท ฟินคับ จำกัด,"90,000 - 120,000 THB",18 ต.ค. 66,3,นิคมอุตสาหกรรมดับบลิวเอชเอ ชลบุรี 2 จ.ชลบุรี,"This role will work with product team members to support the company brand promise of ""uncomprom...","Bachelor’s degree in a relevant domain is desired; however, comparable experience will also be..."
3,Senior Software Engineer (Full Stack)/ Remote/ English Speaking,บริษัท ฟินคับ จำกัด,"90,000 - 120,000 THB",18 ต.ค. 66,4,นิคมอุตสาหกรรมดับบลิวเอชเอ ชลบุรี 2 จ.ชลบุรี,"Our Digital technology is built around a global IoT architecture, with enterprise cloud-based se...",Bachelor’s degree in a technology-related field preferred; equivalent experience will be consi...
4,Full Stack Developer (.Net & React.js),บริษัท มอตโต้ อ๊อคชั่น (ประเทศไทย) จำกัด,"85,000",12 ต.ค. 66,5,เขตประเวศ กรุงเทพมหานคร,We're seeking a skilled Full Stack Engineer with expertise in .NET backend and React.js frontend...,Bachelor's degree in Computer Science or related field. A minimum of 4 years of experience a...
5,Java Developer,บริษัท ไอ แพสชั่น จำกัด,"Up to 85,000 บาท",19 ต.ค. 66,6,"MRT รัชดาภิเษก, สุทธิสาร",มีประสบการณ์การเขียนโปรแกรมคอมพิวเตอร์ด้าน Web Based Application ด้วย Java หรือ J2EE อย่างใดอย...,"Bachelor’s or Master’s Degree in Computer Science, Computer Engineering or other IT-Related f..."
6,Project Coordinator-AOS,ASCGroup,"80,000",19 ต.ค. 66,7,เขตจตุจักร กรุงเทพมหานคร,"- Liaise with business users and developers to define project requirements, scopes, and / object...","Bachelor's degree or higher in Computer Science, IT Engineering, MIS, or related fields. At l..."
7,C#.Net Programmer (สมุทรสาคร),"Nemera Technologies Co, Ltd.","75,000",12 ต.ค. 66,8,อ.เมืองสมุทรสาคร จ.สมุทรสาคร,รายละเอียดของงาน : / พัฒนาระบบใช้งานภายในองค์กร (ERP System) / Get Requirement จาก User แต่ละห...,"ปริญญาตรี, วิทยาการคอมพิวเตอร์ หรือ วิศวกรรมคอมพิวเตอร์ ประสบการณ์พัฒนาระบบ 7 ปีขึ้นไป ASP.N..."
8,Technical Support Manager,"40 Hours recruitment (Thailand) Co., Ltd.","75,000 - 90,000",18 ต.ค. 66,9,เขตดินแดง กรุงเทพมหานคร,- Monitors online security (Infrastructure) for users and networks and takes (95% Internal Use...,"Bachelor’s Degree or higher in Information System, Computer Science, Computer Engineer or rela..."
9,System Analyst,"THE PRODIGY (THAILAND) CO., LTD.","70,000 - 120,000 THB",12 ต.ค. 66,10,เขตคลองเตย กรุงเทพมหานคร,Job Description / / -Coordinate with project manager regarding client/ user’s requirements a...,At least 3 years’ experience as a System Analyst Good knowledge of the SDLC Knowledge in SQL...


In [31]:
job_df2.to_csv('jobthai_019_10.csv', index=False)

In [32]:
# Close the driver when done
driver.quit()